# Breaking the check code

## Strange book:

>Recent advances in interdimensional physics have produced fascinating
>predictions about the fundamentals of our universe!  For example,
>interdimensional physics seems to predict that the universe is, at its root, a
>purely mathematical construct, and that all events are caused by the
>interactions between eight pockets of energy called "registers".
>Furthermore, it seems that while the lower registers primarily control mundane
>things like sound and light, the highest register (the so-called "eighth
>register") is used to control interdimensional events such as teleportation.
>
>A hypothetical such teleportation device would need to have have exactly two
>destinations.  One destination would be used when the eighth register is at its
>minimum energy level - this would be the default operation assuming the user
>has no way to control the eighth register.  In this situation, the teleporter
>should send the user to a preconfigured safe location as a default.
>
>The second destination, however, is predicted to require a very specific
>energy level in the eighth register.  The teleporter must take great care to
>confirm that this energy level is exactly correct before teleporting its user!
>If it is even slightly off, the user would (probably) arrive at the correct
>location, but would briefly experience anomalies in the fabric of reality
>itself - this is, of course, not recommended.  Any teleporter would need to test
>the energy level in the eighth register and abort teleportation if it is not
>exactly correct.
>
>This required precision implies that the confirmation mechanism would be very
>computationally expensive.  While this would likely not be an issue for large-
>scale teleporters, a hypothetical hand-held teleporter would take billions of
>years to compute the result and confirm that the eighth register is correct.
>
>If you find yourself trapped in an alternate dimension with nothing but a
>hand-held teleporter, you will need to extract the confirmation algorithm,
>reimplement it on more powerful hardware, and optimize it.  This should, at the
>very least, allow you to determine the value of the eighth register which would
>have been accepted by the teleporter's confirmation mechanism.
>
>Then, set the eighth register to this value, activate the teleporter, and
>bypass the confirmation mechanism.  If the eighth register is set correctly, no
>anomalies should be experienced, but beware - if it is set incorrectly, the
>now-bypassed confirmation mechanism will not protect you!
>
>Of course, since teleportation is impossible, this is all totally ridiculous.

## Finding the check code

Start to seek every line where register7 is used. There are 4 of those. By debugging the machine we can determin which line is just testing:
* which if register7 is empty on begining
* test when telepert is used (if register7 is 0 normall behaviour)
* passing value of register7 to check function (actually inside of it)
* passing value of register7 to final function which calculating the secret code

## Original code of check function

```
*   6027:      jt  register_0   6035
    6030:     add  register_0   register_1   1
    6034:     ret

    6035:      jt  register_1   6048
    6038:     add  register_0   register_0   32767
    6042:     set  register_1   register_7
    6045:    call  6027
    6047:     ret

    6048:    push  register_0
    6050:     add  register_1   register_1   32767
    6054:    call  6027
    6056:     set  register_1   register_0
    6059:     pop  register_0
    6061:     add  register_0   register_0   32767
    6065:    call  6027
    6067:     ret
```

In [ ]:
import pandas

MAX_INT = 32768

def add32767(value):
    return (value + 32767) % MAX_INT

pandas.DataFrame(
    ({
        'x in function': f'({i} + 32767) % {MAX_INT}',
        'result': add32767(i),
        'approximation': f'({i} - 1) % {MAX_INT}'
    } for i in range(10))
)

In [ ]:
from functools import lru_cache

@lru_cache
def f(x, y, reg7):
    if x == 0:
        return (y + 1) % MAX_INT

    if y == 0:
        return f(x - 1, reg7, reg7)

    return f(x - 1, f(x, y - 1, reg7), reg7)

In [ ]:
paramaters = [(x, y) for x in range(1, 3) for y in range(5)] + [(3, y) for y in range(4)]
test_results = [(x, y, reg7, f(x, y, reg7)) for x, y in paramaters for reg7 in range(10) if not (x == 3 and y == 3 and reg7 > 5)]

out = {}
for test_result in test_results:
    x, y, reg7, result = test_result
    label = f'f({x}, {y}, n)'
    n_result = out.get(label, {})
    n_result[reg7] = result
    out[label] = n_result

pandas.DataFrame(out)

In [ ]:
@lru_cache
def new_f(x, y, reg7):
    if x == 0:
        return (y + 1) % MAX_INT
    if x == 1:
        return (reg7 + 1 + y) % MAX_INT
    if x == 2:
        return ((x + y) * reg7 + (x + y - 1)) % MAX_INT
    if y == 0:
        return new_f(x - 1, reg7, reg7)
   
    return new_f(x - 1, new_f(x, y - 1, reg7), reg7)

def test(x, y, reg7, expected, result):
    assert expected == result, f'Error for {x}, {y}, {reg7}, excepted {expected}, was: {result}'

In [ ]:
for x, y, reg7, expected in test_results:
    if reg7 != 0:
        test(x, y, reg7, expected, new_f(x, y, reg7))    


In [ ]:
print('f(1, y) -> reg7 + 1 + y')
for x, y, reg7, expected in test_results:
    if x == 1:
        test(x, y, reg7, expected, reg7 + 1 + y)

In [ ]:
print('f(2, y) -> (x + y) * reg7 + (x + y - 1)')
for x, y, reg7, expected in test_results:
    if x == 2:
        test(x, y, reg7, expected, (x + y) * reg7 + (x + y - 1))

In [ ]:
approximations = {
    # 1 + n (3 + n)
    (3, 0): lambda n: (n + 1) ** 2 + n, # n^2 + 3 n + 1

    # 2 + n (6 + n (4 + n))
    (3, 1): lambda n: (n + 1) ** 2 * (n + 2) + n, # n^3 + 4 n^2 + 6 n + 2

    # 3 + n (10 + n (10 + n (5 + n)))
    (3, 2): lambda n: (n + 1) ** 2 * (n + 2) + n, # n^4 + 5 n^3 + 10 n^2 + 10 n + 3

    # 4 + n (15 + n (20 + n (15 + n (6 + n))))
    (3, 3): lambda n: (n + 1) ** 4 * (n + 2) + (n + 1)**2 * (n + 2) + n # n^5 + 6 n^4 + 15 n^3 + 20 n^2 + 15 n + 4
}

keys = list(range(len(approximations)))
pandas.DataFrame(
    [[approximations[(3, y)](n) for n in range(10)] for y in keys],
    (f'f(3, {y}, n)' for y in keys))